In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from metal.mmtl.trainer import MultitaskTrainer
from metal.mmtl.glue.glue_tasks import create_glue_tasks_payloads
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.slicing.slice_model import SliceModel

In [3]:
SEED = 1

### Initialize normal payloads

In [4]:
task_kwargs = {
    "dl_kwargs": {"batch_size": 8},
    "freeze_bert":False,
    "bert_model": 'bert-base-cased',
    "max_len": 200,
    "attention": False
}
task_names = ["RTE"]

In [5]:
%%time

# Create tasks and payloads
tasks, payloads = create_glue_tasks_payloads(task_names, **task_kwargs)

Using random seed: 187882
Loading RTE Dataset



CPU times: user 12.2 s, sys: 804 ms, total: 13 s
Wall time: 14.1 s


In [6]:
tasks, payloads

([ClassificationTask(name=RTE, loss_multiplier=1.00)],
 [Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=train),
  Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=valid),
  Payload(RTE_test: labels_to_tasks=[{'RTE_gold': 'RTE'}], split=test)])

### Initialize slice payloads

In [7]:
# Create tasks and payloads
task_kwargs.update({"slice_dict": {
    "RTE": ["dash_semicolon", "more_people", "BASE"]}
})
task_kwargs['attention'] = None

tasks_slice, payloads_slice = create_glue_tasks_payloads(
    task_names, **task_kwargs
)

Using random seed: 365181
Loading RTE Dataset



Added label_set with 2490/2490 labels for task RTE_slice:dash_semicolon:ind to payload RTE_train.
Added label_set with 1003/2490 labels for task RTE_slice:dash_semicolon:pred to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:more_people:ind to payload RTE_train.
Added label_set with 64/2490 labels for task RTE_slice:more_people:pred to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:BASE:ind to payload RTE_train.
Added label_set with 2490/2490 labels for task RTE_slice:BASE:pred to payload RTE_train.
Added label_set with 277/277 labels for task RTE_slice:dash_semicolon:ind to payload RTE_valid.
Added label_set with 116/277 labels for task RTE_slice:dash_semicolon:pred to payload RTE_valid.
Added label_set with 277/277 labels for task RTE_slice:more_people:ind to payload RTE_valid.
Added label_set with 12/277 labels for task RTE_slice:more_people:pred to payload RTE_valid.
Added label_set with 277/277 labels for task RTE_slice:BA

In [8]:
tasks_slice, payloads_slice

([ClassificationTask(name=RTE, loss_multiplier=1.00),
  ClassificationTask(name=RTE_slice:dash_semicolon:ind, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:dash_semicolon:pred, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:more_people:ind, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:more_people:pred, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:BASE:ind, loss_multiplier=0.17),
  ClassificationTask(name=RTE_slice:BASE:pred, loss_multiplier=0.17)],
 [Payload(RTE_train: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolon:ind': 'RTE_slice:dash_semicolon:ind', 'RTE_slice:dash_semicolon:pred': 'RTE_slice:dash_semicolon:pred', 'RTE_slice:more_people:ind': 'RTE_slice:more_people:ind', 'RTE_slice:more_people:pred': 'RTE_slice:more_people:pred', 'RTE_slice:BASE:ind': 'RTE_slice:BASE:ind', 'RTE_slice:BASE:pred': 'RTE_slice:BASE:pred'}], split=train),
  Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolo

In [9]:
payloads_slice[0].data_loader.dataset.labels['RTE_slice:BASE:pred'].view(1, -1)

tensor([[2, 1, 1,  ..., 2, 1, 2]])

In [10]:
payloads_slice[0].data_loader.dataset.labels['RTE_slice:BASE:ind'].view(1, -1)

tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.uint8)

In [11]:
payloads_slice[0].data_loader.dataset.labels['RTE_slice:dash_semicolon:pred'].view(1, -1)

tensor([[0, 0, 0,  ..., 0, 1, 0]])

In [12]:
payloads_slice[0].data_loader.dataset.labels['RTE_slice:dash_semicolon:ind'].view(1, -1)

tensor([[2, 2, 2,  ..., 2, 1, 2]], dtype=torch.uint8)

### Initialize and train baseline model 

In [23]:
model = MetalModel(tasks, seed=SEED, verbose=False)

In [25]:
%%time
trainer = MultitaskTrainer(seed=SEED)
trainer.train_model(
    model,
    payloads,
    checkpoint_metric="RTE/RTE_valid/RTE_gold/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=5e-5,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=10,
    progress_bar=True,
    checkpoint_best=True,
    checkpoint_cleanup=False,
)

Beginning train loop.
Expecting a total of approximately 2496 examples and 312 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/zzweng/metal/logs/2019_04_22/22_40_25/config.json


[0.10 epo]: RTE:[RTE_train/RTE_gold/loss=5.64e-01, RTE_valid/RTE_gold/accuracy=7.11e-01] model:[train/all/loss=5.64e-01, train/all/lr=5.00e-05]
Saving model at iteration 0.10 with best (max) score RTE/RTE_valid/RTE_gold/accuracy=0.711
[0.21 epo]: RTE:[RTE_train/RTE_gold/loss=4.53e-01, RTE_valid/RTE_gold/accuracy=6.97e-01] model:[train/all/loss=4.53e-01, train/all/lr=5.00e-05]
[0.31 epo]: RTE:[RTE_train/RTE_gold/loss=3.76e-01, RTE_valid/RTE_gold/accuracy=7.04e-01] model:[train/all/loss=3.76e-01, train/all/lr=5.00e-05]
[0.41 epo]: RTE:[RTE_train/RTE_gold/loss=2.80e-01, RTE_valid/RTE_gold/accuracy=6.64e-01] model:[train/all/loss=2.80e-01, train/all/lr=5.00e-05]
[0.51 epo]: RTE:[RTE_train/RTE_gold/loss=5.39e-01, RTE_valid/RTE_gold/accuracy=6.57e-01] model:[train/all/loss=5.39e-01, train/all/lr=5.00e-05]
[0.62 epo]: RTE:[RTE_train/RTE_gold/loss=3.61e-01, RTE_valid/RTE_gold/accuracy=6.68e-01] model:[train/all/loss=3.61e-01, train/all/lr=5.00e-05]
[0.72 epo]: RTE:[RTE_train/RTE_gold/loss=3.58

[1.03 epo]: RTE:[RTE_train/RTE_gold/loss=2.81e-01, RTE_valid/RTE_gold/accuracy=6.57e-01] model:[train/all/loss=2.81e-01, train/all/lr=5.00e-05]
[1.13 epo]: RTE:[RTE_train/RTE_gold/loss=2.66e-01, RTE_valid/RTE_gold/accuracy=6.43e-01] model:[train/all/loss=2.66e-01, train/all/lr=5.00e-05]
[1.23 epo]: RTE:[RTE_train/RTE_gold/loss=2.99e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=2.99e-01, train/all/lr=5.00e-05]
[1.33 epo]: RTE:[RTE_train/RTE_gold/loss=2.74e-01, RTE_valid/RTE_gold/accuracy=6.68e-01] model:[train/all/loss=2.74e-01, train/all/lr=5.00e-05]
[1.44 epo]: RTE:[RTE_train/RTE_gold/loss=3.42e-01, RTE_valid/RTE_gold/accuracy=6.82e-01] model:[train/all/loss=3.42e-01, train/all/lr=5.00e-05]
[1.54 epo]: RTE:[RTE_train/RTE_gold/loss=3.50e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=3.50e-01, train/all/lr=5.00e-05]
[1.64 epo]: RTE:[RTE_train/RTE_gold/loss=2.85e-01, RTE_valid/RTE_gold/accuracy=6.90e-01] model:[train/all/loss=2.85e-01, train/all/lr=5.

[2.05 epo]: RTE:[RTE_train/RTE_gold/loss=3.84e-01, RTE_valid/RTE_gold/accuracy=6.46e-01] model:[train/all/loss=3.84e-01, train/all/lr=5.00e-05]
[2.15 epo]: RTE:[RTE_train/RTE_gold/loss=2.55e-01, RTE_valid/RTE_gold/accuracy=6.71e-01] model:[train/all/loss=2.55e-01, train/all/lr=5.00e-05]
[2.26 epo]: RTE:[RTE_train/RTE_gold/loss=2.80e-01, RTE_valid/RTE_gold/accuracy=6.79e-01] model:[train/all/loss=2.80e-01, train/all/lr=5.00e-05]
[2.36 epo]: RTE:[RTE_train/RTE_gold/loss=3.14e-01, RTE_valid/RTE_gold/accuracy=6.46e-01] model:[train/all/loss=3.14e-01, train/all/lr=5.00e-05]
[2.46 epo]: RTE:[RTE_train/RTE_gold/loss=3.48e-01, RTE_valid/RTE_gold/accuracy=6.75e-01] model:[train/all/loss=3.48e-01, train/all/lr=5.00e-05]
[2.56 epo]: RTE:[RTE_train/RTE_gold/loss=3.17e-01, RTE_valid/RTE_gold/accuracy=6.57e-01] model:[train/all/loss=3.17e-01, train/all/lr=5.00e-05]
[2.67 epo]: RTE:[RTE_train/RTE_gold/loss=3.71e-01, RTE_valid/RTE_gold/accuracy=6.57e-01] model:[train/all/loss=3.71e-01, train/all/lr=5.

[3.08 epo]: RTE:[RTE_train/RTE_gold/loss=2.92e-01, RTE_valid/RTE_gold/accuracy=6.79e-01] model:[train/all/loss=2.92e-01, train/all/lr=5.00e-05]
[3.18 epo]: RTE:[RTE_train/RTE_gold/loss=2.47e-01, RTE_valid/RTE_gold/accuracy=6.46e-01] model:[train/all/loss=2.47e-01, train/all/lr=5.00e-05]
[3.28 epo]: RTE:[RTE_train/RTE_gold/loss=2.56e-01, RTE_valid/RTE_gold/accuracy=6.90e-01] model:[train/all/loss=2.56e-01, train/all/lr=5.00e-05]
[3.38 epo]: RTE:[RTE_train/RTE_gold/loss=2.99e-01, RTE_valid/RTE_gold/accuracy=6.43e-01] model:[train/all/loss=2.99e-01, train/all/lr=5.00e-05]
[3.49 epo]: RTE:[RTE_train/RTE_gold/loss=4.34e-01, RTE_valid/RTE_gold/accuracy=6.64e-01] model:[train/all/loss=4.34e-01, train/all/lr=5.00e-05]
[3.59 epo]: RTE:[RTE_train/RTE_gold/loss=3.90e-01, RTE_valid/RTE_gold/accuracy=6.61e-01] model:[train/all/loss=3.90e-01, train/all/lr=5.00e-05]
[3.69 epo]: RTE:[RTE_train/RTE_gold/loss=3.41e-01, RTE_valid/RTE_gold/accuracy=6.28e-01] model:[train/all/loss=3.41e-01, train/all/lr=5.

...

[5.03 epo]: RTE:[RTE_train/RTE_gold/loss=4.63e-01, RTE_valid/RTE_gold/accuracy=6.53e-01] model:[train/all/loss=4.63e-01, train/all/lr=5.00e-05]
[5.13 epo]: RTE:[RTE_train/RTE_gold/loss=2.19e-01, RTE_valid/RTE_gold/accuracy=6.50e-01] model:[train/all/loss=2.19e-01, train/all/lr=5.00e-05]
[5.23 epo]: RTE:[RTE_train/RTE_gold/loss=1.96e-01, RTE_valid/RTE_gold/accuracy=6.32e-01] model:[train/all/loss=1.96e-01, train/all/lr=5.00e-05]
[5.33 epo]: RTE:[RTE_train/RTE_gold/loss=3.98e-01, RTE_valid/RTE_gold/accuracy=6.39e-01] model:[train/all/loss=3.98e-01, train/all/lr=5.00e-05]
[5.44 epo]: RTE:[RTE_train/RTE_gold/loss=3.84e-01, RTE_valid/RTE_gold/accuracy=6.43e-01] model:[train/all/loss=3.84e-01, train/all/lr=5.00e-05]
[5.54 epo]: RTE:[RTE_train/RTE_gold/loss=2.60e-01, RTE_valid/RTE_gold/accuracy=6.10e-01] model:[train/all/loss=2.60e-01, train/all/lr=5.00e-05]
[5.64 epo]: RTE:[RTE_train/RTE_gold/loss=3.38e-01, RTE_valid/RTE_gold/accuracy=6.21e-01] model:[train/all/loss=3.38e-01, train/all/lr=5.

[6.05 epo]: RTE:[RTE_train/RTE_gold/loss=2.95e-01, RTE_valid/RTE_gold/accuracy=6.28e-01] model:[train/all/loss=2.95e-01, train/all/lr=5.00e-05]
[6.15 epo]: RTE:[RTE_train/RTE_gold/loss=2.42e-01, RTE_valid/RTE_gold/accuracy=5.85e-01] model:[train/all/loss=2.42e-01, train/all/lr=5.00e-05]
[6.26 epo]: RTE:[RTE_train/RTE_gold/loss=2.76e-01, RTE_valid/RTE_gold/accuracy=6.25e-01] model:[train/all/loss=2.76e-01, train/all/lr=5.00e-05]
[6.36 epo]: RTE:[RTE_train/RTE_gold/loss=4.30e-01, RTE_valid/RTE_gold/accuracy=6.14e-01] model:[train/all/loss=4.30e-01, train/all/lr=5.00e-05]
[6.46 epo]: RTE:[RTE_train/RTE_gold/loss=3.15e-01, RTE_valid/RTE_gold/accuracy=6.21e-01] model:[train/all/loss=3.15e-01, train/all/lr=5.00e-05]
[6.56 epo]: RTE:[RTE_train/RTE_gold/loss=3.19e-01, RTE_valid/RTE_gold/accuracy=6.03e-01] model:[train/all/loss=3.19e-01, train/all/lr=5.00e-05]
[6.67 epo]: RTE:[RTE_train/RTE_gold/loss=4.13e-01, RTE_valid/RTE_gold/accuracy=6.03e-01] model:[train/all/loss=4.13e-01, train/all/lr=5.

[7.08 epo]: RTE:[RTE_train/RTE_gold/loss=2.43e-01, RTE_valid/RTE_gold/accuracy=6.75e-01] model:[train/all/loss=2.43e-01, train/all/lr=5.00e-05]
[7.18 epo]: RTE:[RTE_train/RTE_gold/loss=4.64e-01, RTE_valid/RTE_gold/accuracy=6.35e-01] model:[train/all/loss=4.64e-01, train/all/lr=5.00e-05]
[7.28 epo]: RTE:[RTE_train/RTE_gold/loss=3.82e-01, RTE_valid/RTE_gold/accuracy=5.85e-01] model:[train/all/loss=3.82e-01, train/all/lr=5.00e-05]
[7.38 epo]: RTE:[RTE_train/RTE_gold/loss=3.06e-01, RTE_valid/RTE_gold/accuracy=6.46e-01] model:[train/all/loss=3.06e-01, train/all/lr=5.00e-05]
[7.49 epo]: RTE:[RTE_train/RTE_gold/loss=3.80e-01, RTE_valid/RTE_gold/accuracy=6.28e-01] model:[train/all/loss=3.80e-01, train/all/lr=5.00e-05]
[7.59 epo]: RTE:[RTE_train/RTE_gold/loss=3.24e-01, RTE_valid/RTE_gold/accuracy=6.35e-01] model:[train/all/loss=3.24e-01, train/all/lr=5.00e-05]
[7.69 epo]: RTE:[RTE_train/RTE_gold/loss=4.54e-01, RTE_valid/RTE_gold/accuracy=6.43e-01] model:[train/all/loss=4.54e-01, train/all/lr=5.

[8.10 epo]: RTE:[RTE_train/RTE_gold/loss=2.48e-01, RTE_valid/RTE_gold/accuracy=6.06e-01] model:[train/all/loss=2.48e-01, train/all/lr=5.00e-05]
[8.21 epo]: RTE:[RTE_train/RTE_gold/loss=3.48e-01, RTE_valid/RTE_gold/accuracy=5.85e-01] model:[train/all/loss=3.48e-01, train/all/lr=5.00e-05]
[8.31 epo]: RTE:[RTE_train/RTE_gold/loss=3.05e-01, RTE_valid/RTE_gold/accuracy=5.88e-01] model:[train/all/loss=3.05e-01, train/all/lr=5.00e-05]
[8.41 epo]: RTE:[RTE_train/RTE_gold/loss=4.28e-01, RTE_valid/RTE_gold/accuracy=6.21e-01] model:[train/all/loss=4.28e-01, train/all/lr=5.00e-05]
[8.51 epo]: RTE:[RTE_train/RTE_gold/loss=2.77e-01, RTE_valid/RTE_gold/accuracy=6.03e-01] model:[train/all/loss=2.77e-01, train/all/lr=5.00e-05]
[8.62 epo]: RTE:[RTE_train/RTE_gold/loss=2.85e-01, RTE_valid/RTE_gold/accuracy=5.99e-01] model:[train/all/loss=2.85e-01, train/all/lr=5.00e-05]
[8.72 epo]: RTE:[RTE_train/RTE_gold/loss=5.44e-01, RTE_valid/RTE_gold/accuracy=6.03e-01] model:[train/all/loss=5.44e-01, train/all/lr=5.

[9.03 epo]: RTE:[RTE_train/RTE_gold/loss=3.84e-01, RTE_valid/RTE_gold/accuracy=6.03e-01] model:[train/all/loss=3.84e-01, train/all/lr=5.00e-05]
[9.13 epo]: RTE:[RTE_train/RTE_gold/loss=2.92e-01, RTE_valid/RTE_gold/accuracy=5.81e-01] model:[train/all/loss=2.92e-01, train/all/lr=5.00e-05]
[9.23 epo]: RTE:[RTE_train/RTE_gold/loss=3.08e-01, RTE_valid/RTE_gold/accuracy=5.74e-01] model:[train/all/loss=3.08e-01, train/all/lr=5.00e-05]
[9.33 epo]: RTE:[RTE_train/RTE_gold/loss=3.18e-01, RTE_valid/RTE_gold/accuracy=5.85e-01] model:[train/all/loss=3.18e-01, train/all/lr=5.00e-05]
[9.44 epo]: RTE:[RTE_train/RTE_gold/loss=3.92e-01, RTE_valid/RTE_gold/accuracy=6.21e-01] model:[train/all/loss=3.92e-01, train/all/lr=5.00e-05]
[9.54 epo]: RTE:[RTE_train/RTE_gold/loss=3.43e-01, RTE_valid/RTE_gold/accuracy=6.06e-01] model:[train/all/loss=3.43e-01, train/all/lr=5.00e-05]
[9.64 epo]: RTE:[RTE_train/RTE_gold/loss=5.63e-01, RTE_valid/RTE_gold/accuracy=6.06e-01] model:[train/all/loss=5.63e-01, train/all/lr=5.

### Evaluate baseline slices

In [15]:
import copy
eval_payload = copy.deepcopy(payloads_slice[1])

# NOTE: we need to retarget slices to the original RTE head
for label_name in ['RTE_slice:dash_semicolon', 'RTE_slice:more_people', 'RTE_slice:BASE']:
    for slice_head_type in ['ind', 'pred']:
        eval_payload.retarget_labelset(f'{label_name}:{slice_head_type}', 'RTE')

labelset 'RTE_slice:dash_semicolon:ind' -> task 'RTE' (originally, RTE_slice:dash_semicolon:ind).
labelset 'RTE_slice:dash_semicolon:pred' -> task 'RTE' (originally, RTE_slice:dash_semicolon:pred).
labelset 'RTE_slice:more_people:ind' -> task 'RTE' (originally, RTE_slice:more_people:ind).
labelset 'RTE_slice:more_people:pred' -> task 'RTE' (originally, RTE_slice:more_people:pred).
labelset 'RTE_slice:BASE:ind' -> task 'RTE' (originally, RTE_slice:BASE:ind).
labelset 'RTE_slice:BASE:pred' -> task 'RTE' (originally, RTE_slice:BASE:pred).


In [16]:
eval_payload

Payload(RTE_valid: labels_to_tasks=[{'RTE_gold': 'RTE', 'RTE_slice:dash_semicolon:ind': 'RTE', 'RTE_slice:dash_semicolon:pred': 'RTE', 'RTE_slice:more_people:ind': 'RTE', 'RTE_slice:more_people:pred': 'RTE', 'RTE_slice:BASE:ind': 'RTE', 'RTE_slice:BASE:pred': 'RTE'}], split=valid)

In [17]:
model.score(eval_payload)

### Initialize and train slice model

In [18]:
from metal.mmtl.slicing.tasks import convert_to_slicing_tasks
slicing_tasks = convert_to_slicing_tasks(tasks_slice)
slicing_tasks

Modifying RTE out_features from 2 -> 1
Modifying RTE_slice:dash_semicolon:ind out_features from 2 -> 1
Modifying RTE_slice:dash_semicolon:pred out_features from 2 -> 1
Modifying RTE_slice:more_people:ind out_features from 2 -> 1
Modifying RTE_slice:more_people:pred out_features from 2 -> 1
Modifying RTE_slice:BASE:ind out_features from 2 -> 1
Modifying RTE_slice:BASE:pred out_features from 2 -> 1


[BinaryClassificationTask(name=RTE, loss_multiplier=1.00, slice_head_type=None),
 BinaryClassificationTask(name=RTE_slice:dash_semicolon:ind, loss_multiplier=0.17, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:dash_semicolon:pred, loss_multiplier=0.17, slice_head_type=pred),
 BinaryClassificationTask(name=RTE_slice:more_people:ind, loss_multiplier=0.17, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:more_people:pred, loss_multiplier=0.17, slice_head_type=pred),
 BinaryClassificationTask(name=RTE_slice:BASE:ind, loss_multiplier=0.17, slice_head_type=ind),
 BinaryClassificationTask(name=RTE_slice:BASE:pred, loss_multiplier=0.17, slice_head_type=pred)]

In [19]:
slice_model = SliceModel(slicing_tasks, seed=SEED, verbose=False)

In [20]:
## %%time
trainer = MultitaskTrainer(seed=SEED)
trainer.train_model(
    slice_model,
    payloads_slice,
    task_metrics=[
        "RTE/RTE_train/RTE_gold/loss", 
        "RTE/RTE_train/RTE_slice:BASE:ind/loss",
        "RTE/RTE_train/RTE_slice:dash_semicolon:ind/loss", 
        "RTE/RTE_train/RTE_slice:more_people:ind/loss",
        "RTE/RTE_train/RTE_slice:BASE:pred/loss",
        "RTE/RTE_train/RTE_slice:dash_semicolon:pred/loss", 
        "RTE/RTE_train/RTE_slice:more_people:pred/loss",
        
        "RTE/RTE_valid/RTE_gold/accuracy",
        "RTE/RTE_valid/RTE_slice:BASE:ind/accuracy", 
        "RTE/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy", 
        "RTE/RTE_valid/RTE_slice:more_people:ind/accuracy",
        "RTE/RTE_valid/RTE_slice:BASE:pred/accuracy", 
        "RTE/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy", 
        "RTE/RTE_valid/RTE_slice:more_people:pred/accuracy",
    ],
    checkpoint_metric="RTE/RTE_valid/RTE_gold/accuracy",
    checkpoint_metric_mode="max",
    checkoint_best=True,
    writer="tensorboard",
    optimizer="adamax",
    lr=1e-5,
#     l2=1e-3,
    l2=1e-3,
    log_every=0.1, 
    score_every=0.1,
    n_epochs=20,
    progress_bar=True,
    checkpoint_best=True,
    checkpoint_cleanup=False
)

Beginning train loop.
Expecting a total of approximately 2496 examples and 312 batches per epoch from 1 payload(s) in the train split.
Writing config to /dfs/scratch0/zzweng/metal/logs/2019_04_22/21_42_56/config.json


[0.10 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=4.40e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.20e-01, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=4.69e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=1.16e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.26e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=6.94e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=1.14e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=1.60e-01, RTE_valid/RTE_slice:BASE:ind/accuracy=0.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=1.20e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=4.73e-01] model:[train/all/loss=2.24e-01, train/all/lr=1.00e-05]
Saving model at iteration 0.10 with best (max) score RTE/RTE_valid/RTE_go

[1.03 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=4.69e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.14e-01, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=5.85e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=1.16e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.43e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=3.02e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=3.07e-02, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=1.14e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=4.91e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=1.19e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] model:[train/all/loss=1.90e-01, train/all/lr=1.00e-05]
[1.13 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/acc

[1.95 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=4.77e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.10e-01, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=5.81e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=1.15e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.52e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=2.17e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=1.05e-02, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=1.14e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=5.78e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=1.01e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] model:[train/all/loss=1.85e-01, train/all/lr=1.00e-05]
Saving model at iteration 1.95 with best (max) score RTE/RTE_valid/RTE_go

[2.05 epo]: RTE:[RTE_train/RTE_gold/loss=6.93e-01, RTE_valid/RTE_gold/accuracy=4.80e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.08e-01, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=5.81e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=1.14e-01, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.69e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=1.51e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=9.53e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=1.13e-01, RTE_valid/RTE_slice:BASE:pred/accuracy=6.03e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=1.26e-01, RTE_valid/RTE_slice:more_people:pred/accuracy=4.17e-01] model:[train/all/loss=1.83e-01, train/all/lr=1.00e-05]
Saving model at iteration 2.05 with best (max) score RTE/RTE_valid/RTE_go

...

[18.05 epo]: RTE:[RTE_train/RTE_gold/loss=6.62e-01, RTE_valid/RTE_gold/accuracy=6.82e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.41e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.82e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=1.53e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=2.12e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=3.47e-02, RTE_valid/RTE_slice:BASE:pred/accuracy=6.82e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=1.49e-02, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=6.55e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=2.64e-02, RTE_valid/RTE_slice:more_people:pred/accuracy=6.67e-01] model:[train/all/loss=1.37e-01, train/all/lr=1.00e-05]
[18.15 epo]: RTE:[RTE_train/RTE_gold/loss=6.61e-01, RTE_valid/RTE_gold/a

[18.97 epo]: RTE:[RTE_train/RTE_gold/loss=6.58e-01, RTE_valid/RTE_gold/accuracy=6.43e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.23e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.82e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=1.22e-02, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.95e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=2.56e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=2.34e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=3.25e-02, RTE_valid/RTE_slice:BASE:pred/accuracy=6.43e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=2.89e-02, RTE_valid/RTE_slice:more_people:pred/accuracy=7.50e-01] model:[train/all/loss=1.36e-01, train/all/lr=1.00e-05]



[19.08 epo]: RTE:[RTE_train/RTE_gold/loss=6.58e-01, RTE_valid/RTE_gold/accuracy=6.53e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.11e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.82e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=1.62e-02, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=6.47e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=9.92e-03, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=2.59e-02, RTE_valid/RTE_slice:more_people:pred/accuracy=3.33e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=2.07e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=3.63e-02, RTE_valid/RTE_slice:BASE:pred/accuracy=6.64e-01] model:[train/all/loss=1.36e-01, train/all/lr=1.00e-05]
[19.18 epo]: RTE:[RTE_train/RTE_gold/loss=6.56e-01, RTE_valid/RTE_gold/a

[20.0 epo]: RTE:[RTE_train/RTE_gold/loss=6.52e-01, RTE_valid/RTE_gold/accuracy=6.35e-01] RTE_slice:dash_semicolon:ind:[RTE_train/RTE_slice:dash_semicolon:ind/loss=1.15e-02, RTE_valid/RTE_slice:dash_semicolon:ind/accuracy=9.93e-01] RTE_slice:dash_semicolon:pred:[RTE_train/RTE_slice:dash_semicolon:pred/loss=2.77e-02, RTE_valid/RTE_slice:dash_semicolon:pred/accuracy=5.86e-01] RTE_slice:more_people:ind:[RTE_train/RTE_slice:more_people:ind/loss=2.08e-02, RTE_valid/RTE_slice:more_people:ind/accuracy=9.57e-01] RTE_slice:more_people:pred:[RTE_train/RTE_slice:more_people:pred/loss=3.12e-02, RTE_valid/RTE_slice:more_people:pred/accuracy=2.50e-01] RTE_slice:BASE:ind:[RTE_train/RTE_slice:BASE:ind/loss=2.46e-03, RTE_valid/RTE_slice:BASE:ind/accuracy=1.00e+00] RTE_slice:BASE:pred:[RTE_train/RTE_slice:BASE:pred/loss=3.15e-02, RTE_valid/RTE_slice:BASE:pred/accuracy=6.43e-01] model:[train/all/loss=1.35e-01, train/all/lr=1.00e-05]

[20.00 epo]: model:[train/all/lr=1.00e-05] RTE:[RTE_valid/RTE_gold/accur

{'RTE/RTE_train/RTE_gold/accuracy': 0.9654618473895582,
 'RTE_slice:dash_semicolon:ind/RTE_train/RTE_slice:dash_semicolon:ind/accuracy': 0.9919678714859438,
 'RTE_slice:dash_semicolon:pred/RTE_train/RTE_slice:dash_semicolon:pred/accuracy': 0.9900299102691924,
 'RTE_slice:more_people:ind/RTE_train/RTE_slice:more_people:ind/accuracy': 0.9759036144578314,
 'RTE_slice:more_people:pred/RTE_train/RTE_slice:more_people:pred/accuracy': 0.984375,
 'RTE_slice:BASE:ind/RTE_train/RTE_slice:BASE:ind/accuracy': 0.9991967871485944,
 'RTE_slice:BASE:pred/RTE_train/RTE_slice:BASE:pred/accuracy': 0.9646586345381526,
 'RTE/RTE_valid/RTE_gold/accuracy': 0.7148014440433214,
 'RTE_slice:dash_semicolon:ind/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 0.9819494584837545,
 'RTE_slice:dash_semicolon:pred/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.6551724137931034,
 'RTE_slice:more_people:ind/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.9566787003610109,
 'RTE_slice:more_people:pred/RTE_valid/

#### Did we improve?

In [21]:
%%time
slice_model.score(payloads_slice[1])

CPU times: user 1.62 s, sys: 164 ms, total: 1.79 s
Wall time: 1.76 s


{'RTE/RTE_valid/RTE_gold/accuracy': 0.7148014440433214,
 'RTE_slice:dash_semicolon:ind/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 0.9819494584837545,
 'RTE_slice:dash_semicolon:pred/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.6551724137931034,
 'RTE_slice:more_people:ind/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.9566787003610109,
 'RTE_slice:more_people:pred/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.6666666666666666,
 'RTE_slice:BASE:ind/RTE_valid/RTE_slice:BASE:ind/accuracy': 1.0,
 'RTE_slice:BASE:pred/RTE_valid/RTE_slice:BASE:pred/accuracy': 0.7111913357400722}

In [22]:
slice_model.score(eval_payload)

{'RTE/RTE_valid/RTE_gold/accuracy': 0.7148014440433214,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:ind/accuracy': 0.51985559566787,
 'RTE/RTE_valid/RTE_slice:dash_semicolon:pred/accuracy': 0.6724137931034483,
 'RTE/RTE_valid/RTE_slice:more_people:ind/accuracy': 0.4981949458483754,
 'RTE/RTE_valid/RTE_slice:more_people:pred/accuracy': 0.8333333333333334,
 'RTE/RTE_valid/RTE_slice:BASE:ind/accuracy': 0.48014440433212996,
 'RTE/RTE_valid/RTE_slice:BASE:pred/accuracy': 0.7148014440433214}